In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf, SQLContext

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

CLICKHOUSE_IP = "34.32.47.131"
CLICKHOUSE_PORT = 9000
CLICKHOUSE_USER = "default"
CLICKHOUSE_USER_PASSWORD = "12341"

#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0-patch5",
    "com.clickhouse:clickhouse-http-client:0.6.0-patch5",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1",
    "ai.catboost:catboost-spark_3.5_2.12:1.2.5"
]
ram = 30
cpu = 22*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.jars.packages", ",".join(packages))
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", CLICKHOUSE_IP)
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", CLICKHOUSE_USER)
         .config("spark.sql.catalog.clickhouse.password", CLICKHOUSE_USER_PASSWORD)
         .config("spark.sql.catalog.clickhouse.database", "default")
         #.config("spark.spark.clickhouse.write.compression.codec", "lz4")
         #.config("spark.clickhouse.read.compression.codec", "lz4")
         #.config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") 
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
         #.config("spark.clickhouse.write.batchSize", "1000000")
         #.config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
         #  .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
         #  .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
         #  .config("spark.clickhouse.options.socket_timeout", "600000000")
         #  .config("spark.clickhouse.options.connection_timeout", "600000000")
         #  .config("spark.clickhouse.options.query_timeout", "600000000")         
         .config("spark.executor.memory", f"{ram}g")
         #.config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         #.config("spark.driver.memory", f"{ram}g")
         #.config("spark.executor.memoryOverhead", f"{ram}g")
         #.config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
#SedonaRegistrator.registerAll(spark)
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
spark.sql("use clickhouse")

from ydata_profiling import ProfileReport
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import mlflow
import optuna


root_path = "/app"
#root_path = "."
#path_data = f'{root_path}/data'
your_mlflow_tracking_uri = f'{root_path}/mlruns'

mlflow.set_tracking_uri(your_mlflow_tracking_uri)

import warnings
warnings.filterwarnings("ignore")

from optuna.integration.mlflow import MLflowCallback

import catboost_spark

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.github.housepower#clickhouse-spark-runtime-3.4_2.12 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
com.github.housepower#clickhouse-native-jdbc added as a dependency
ai.catboost#catboost-spark_3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6bbef374-d7ab-49c5-a555-8e58c3ba142d;1.0
	confs: [default]
	found com.github.housepower#clickhouse-spark-runtime-3.4_2.12;0.7.3 in central
	found com.clickhouse#clickhouse-jdbc;0.6.0-patch5 in central
	found com.clickhouse#clickhouse-http-client;0.6.0-patch5 in central
	found com.clickhouse#clickhouse-client;0.6.0-patch5 in central
	found com.clickhouse#clickhouse-data;0.6.0-patch5 in central
	found org.apache.httpcomponents.client5#httpclient5;5.3

In [23]:
",".join(packages)

'com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3,com.clickhouse:clickhouse-jdbc:0.6.0-patch5,com.clickhouse:clickhouse-http-client:0.6.0-patch5,org.apache.httpcomponents.client5:httpclient5:5.3.1,com.github.housepower:clickhouse-native-jdbc:2.7.1'

In [21]:
packages

['com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3',
 'com.clickhouse:clickhouse-jdbc:0.6.0-patch5',
 'com.clickhouse:clickhouse-http-client:0.6.0-patch5',
 'org.apache.httpcomponents.client5:httpclient5:5.3.1',
 'com.github.housepower:clickhouse-native-jdbc:2.7.1']

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import col
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import when
import requests

In [18]:
# create a catboost pipeline
!pip install pyspark catboost





In [3]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("CatBoostPipeline") \
    .getOrCreate()


24/06/15 08:49:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# Load dataset
# relative path
data = spark.read.csv('files/bank-additional-full.csv', header=True, inferSchema=True, sep=';')
data.show()



+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57|   services| married|        high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.

In [6]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- y: string (nullable = true)


In [5]:
#Write a code for count nulls using pyspark
from pyspark.sql.functions import isnan, when, count, col
data = (data.withColumnRenamed("emp.var.rate", "emp_var_rate")
        .withColumnRenamed("cons.price.idx", "cons_price_idx")
        .withColumnRenamed("cons.conf.idx", "cons_conf_idx")
        .withColumnRenamed("nr.employed", "nr_employed"))
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()



+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|age|job|marital|education|default|housing|loan|contact|month|day_of_week|duration|campaign|pdays|previous|poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|  0|  0|      0|        0|      0|      0|   0|      0|    0|          0|       0|       0|    0|       0|       0|           0|             0|            0|        0|          0|  0|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+


In [24]:
data.select([when(isnan(c) | col(c).isNull(), c).otherwise(1) for c in data.columns]).show()

+-----------------------------------------------------------+-----------------------------------------------------------+-----------------------------------------------------------------------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------+-----------------------------------------------------------------------+--------------------------------------------------------------+-----------------------------------------------------------------------+-----------------------------------------------------------------+-----------------------------------------------------------------------------------+--------------------------------------------------------------------------+--------------------------------------------------------------------------+-----------------------------------------------------------------+--------------------------------------------------------------------------+---------

In [54]:
data.select([F.sum(when(isnan(c) | col(c).isNull(), 1).otherwise(0)).alias(c) for c in data.columns]).show()

+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|age|job|marital|education|default|housing|loan|contact|month|day_of_week|duration|campaign|pdays|previous|poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|  0|  0|      0|        0|      0|      0|   0|      0|    0|          0|       0|       0|    0|       0|       0|           0|             0|            0|        0|          0|  0|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+


In [49]:
[when(isnan(c) | col(c).isNull(), 1).otherwise(0) for c in data.columns]

[Column<'CASE WHEN (isnan(age) OR (age IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(job) OR (job IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(marital) OR (marital IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(education) OR (education IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(default) OR (default IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(housing) OR (housing IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(loan) OR (loan IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(contact) OR (contact IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(month) OR (month IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(day_of_week) OR (day_of_week IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(duration) OR (duration IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(campaign) OR (campaign IS NULL)) THEN 1 ELSE 0 END'>,
 Column<'CASE WHEN (isnan(pdays) OR (pdays IS NULL)) THEN 1 ELSE 0 END'>,


In [22]:
[c + "_1" if "e" in c else c + "_2" for c in data.columns]\
                                    

['age_1',
 'job_2',
 'marital_2',
 'education_1',
 'default_1',
 'housing_2',
 'loan_2',
 'contact_2',
 'month_2',
 'day_of_week_1',
 'duration_2',
 'campaign_2',
 'pdays_2',
 'previous_1',
 'poutcome_1',
 'emp_var_rate_1',
 'cons_price_idx_1',
 'cons_conf_idx_2',
 'euribor3m_1',
 'nr_employed_1',
 'y_2']

In [23]:
[c + "_1" for c in data.columns if "e" in c]

['age_1',
 'education_1',
 'default_1',
 'day_of_week_1',
 'previous_1',
 'poutcome_1',
 'emp_var_rate_1',
 'cons_price_idx_1',
 'euribor3m_1',
 'nr_employed_1']

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from catboost import CatBoostClassifier, Pool

# Create a Spark session
spark = SparkSession.builder \
    .appName("CatBoostPipeline") \
    .getOrCreate()

# Load dataset

df = spark.read.csv('files/bank-additional-full.csv', header=True, inferSchema=True, sep=';')

# Assuming the last column is the target
target_column = df.columns[-1]
feature_columns = df.columns[:-1]

# Index categorical features
indexers = [StringIndexer(inputCol=column, outputCol=column+"_indexed").fit(df) for column in df.columns if df.schema[column].dataType == 'StringType']
indexing_pipeline = Pipeline(stages=indexers)
df = indexing_pipeline.fit(df).transform(df)

# Assemble features into a single vector
assembler = VectorAssembler(
    inputCols=[column+"_indexed" if df.schema[column].dataType == 'StringType' else column for column in feature_columns],
    outputCol="features"
)
df = assembler.transform(df)

# Convert Spark DataFrame to CatBoost Pool
def spark_df_to_catboost_pool(spark_df, label_column, feature_column):
    pandas_df = spark_df.select([label_column, feature_column]).toPandas()
    labels = pandas_df[label_column].values
    features = pandas_df[feature_column].apply(lambda x: x.toArray()).values.tolist()
    return Pool(data=features, label=labels)

catboost_train_pool = spark_df_to_catboost_pool(df, target_column, "features")

# Define the CatBoost model
catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    verbose=0
)

# Train the CatBoost model
catboost_model.fit(catboost_train_pool)

# Make predictions
predictions = catboost_model.predict(df.select("features").rdd.map(lambda row: row[0].toArray().tolist()).collect())

# Convert predictions to Spark DataFrame
predictions_df = spark.createDataFrame([(float(pred),) for pred in predictions], ["prediction"])

# Combine predictions with original DataFrame
df = df.withColumn("prediction", predictions_df["prediction"])

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(df)
print(f"Accuracy: {accuracy}")

# Stop the Spark session
spark.stop()


IllegalArgumentException: Data type string of column job is not supported.
Data type string of column marital is not supported.
Data type string of column education is not supported.
Data type string of column default is not supported.
Data type string of column housing is not supported.
Data type string of column loan is not supported.
Data type string of column contact is not supported.
Data type string of column month is not supported.
Data type string of column day_of_week is not supported.
Data type string of column poutcome is not supported.

In [2]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from catboost_spark import SparkCatboostClassifier


# Загрузка данных
data = spark.read.csv('files/bank-additional-full.csv', header=True, inferSchema=True, sep=';')

# Определение категориальных столбцов
categorical_features = [
    'job', 'marital', 'education', 'default',
    'housing', 'loan', 'contact', 'month',
    'day_of_week', 'poutcome'
]

# Конвертация столбцов в категориальные
#for col in categorical_features:
    #data[col] = data[col].astype('category')

# Разделение данных на признаки и целевую переменную
X = data.drop('y')  # Замените 'target_column' на имя вашей целевой переменной
y = data['y']

# Создание пула данных для CatBoost
train_data = Pool(X, label=y, cat_features=categorical_features)

# Создание и обучение модели
model = CatBoostClassifier(iterations=10, learning_rate=0.1, depth=6)
model.fit(train_data)

# Пример предсказания
predictions = model.predict(X)
print(predictions)


ModuleNotFoundError: No module named 'catboost.spark'

In [12]:
import pandas as pd
from catboost import CatBoostClassifier, Pool

# Загрузка данных
data = pd.read_csv('files/bank-additional-full.csv', sep=';')

# Определение категориальных столбцов
categorical_features = [
    'job', 'marital', 'education', 'default',
    'housing', 'loan', 'contact', 'month',
    'day_of_week', 'poutcome'
]

# Конвертация столбцов в категориальные
for col in categorical_features:
    data[col] = data[col].astype('category')

# Разделение данных на признаки и целевую переменную
X = data.drop('y', axis=1)  # Замените 'target_column' на имя вашей целевой переменной
y = data['y']

# Создание пула данных для CatBoost
train_data = Pool(data=X, label=y, cat_features=categorical_features)

# Создание и обучение модели
model = CatBoostClassifier()
model.fit(train_data)

# Пример предсказания
predictions = model.predict(X)
print(predictions)


Learning rate set to 0.050401
0:	learn: 0.6064590	total: 108ms	remaining: 1m 47s
1:	learn: 0.5439976	total: 232ms	remaining: 1m 55s
2:	learn: 0.4939144	total: 351ms	remaining: 1m 56s
3:	learn: 0.4502783	total: 461ms	remaining: 1m 54s
4:	learn: 0.4035291	total: 579ms	remaining: 1m 55s
5:	learn: 0.3730118	total: 687ms	remaining: 1m 53s
6:	learn: 0.3435747	total: 800ms	remaining: 1m 53s
7:	learn: 0.3241333	total: 912ms	remaining: 1m 53s
8:	learn: 0.3055240	total: 1.01s	remaining: 1m 51s
9:	learn: 0.2897273	total: 1.13s	remaining: 1m 52s
10:	learn: 0.2779728	total: 1.25s	remaining: 1m 52s
11:	learn: 0.2672690	total: 1.38s	remaining: 1m 53s
12:	learn: 0.2577780	total: 1.49s	remaining: 1m 53s
13:	learn: 0.2482721	total: 1.61s	remaining: 1m 53s
14:	learn: 0.2430648	total: 1.73s	remaining: 1m 53s
15:	learn: 0.2362288	total: 1.83s	remaining: 1m 52s
16:	learn: 0.2301060	total: 1.94s	remaining: 1m 52s
17:	learn: 0.2262609	total: 2.05s	remaining: 1m 51s
18:	learn: 0.2205400	total: 2.16s	remaining: